In [1]:
import pandas as pd
import qiime2 as q2

In [2]:
md = pd.read_csv('/home/lakhatib/thdmi/data_cleaning/1_genome_coverage/thdmi_metadata.tsv', sep = '\t', dtype={'#SampleID': str}).set_index('#SampleID')

/tmp/ipykernel_252932/1821422975.py:1: DtypeWarning: Columns (311,492,532,866) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv('/home/lakhatib/thdmi/data_cleaning/1_genome_coverage/thdmi_metadata.tsv', sep = '\t', dtype={'#SampleID': str}).set_index('#SampleID')


In [ ]:
ft = pd.read_csv('/home/lakhatib/thdmi/data/zebra_feature-table.tsv', sep = '\t', dtype={'Unnamed: 0': str}).set_index('Unnamed: 0')

In [ ]:
print(f"Total Samples: {len(md)}")
print(md['thdmi_cohort'].value_counts())

Total Samples: 2747
Mexico            549
US                534
Japan             524
Spain             481
UK                366
not applicable    277
unknown            10
control sample      6
Name: thdmi_cohort, dtype: int64


In [ ]:
#filter to valid countries
md = md.loc[md['thdmi_cohort'].isin(['Mexico', 'US', 'UK'])]

In [ ]:
print(f"Total Samples: {len(md)}")
print(md['thdmi_cohort'].value_counts())

Total Samples: 1449
Mexico    549
US        534
UK        366
Name: thdmi_cohort, dtype: int64


In [73]:
md = md.loc[md.keep_sample_for_thdmi == 'yes']

In [74]:
print(f"Total Samples: {len(md)}")
print(md['thdmi_cohort'].value_counts())

Total Samples: 1291
Mexico    507
US        442
UK        342
Name: thdmi_cohort, dtype: int64


In [75]:
md = md.loc[md.index.intersection(ft.index)]

In [76]:
print(f"Total Samples: {len(md)}")
print(md['thdmi_cohort'].value_counts())

Total Samples: 1291
Mexico    507
US        442
UK        342
Name: thdmi_cohort, dtype: int64


In [77]:
#drop samples with invalid covariates
import numpy as np
md['host_body_mass_index'] = md['host_body_mass_index'].replace('not provided', np.nan)

md['host_body_mass_index'] = md['host_body_mass_index'].astype(float)

md['host_body_mass_index'] = np.where((md['host_body_mass_index'] < 12) | (md['host_body_mass_index'] > 70), np.nan, 
                                            md['host_body_mass_index']) 

md['covid_level_of_wellbeing'] = md['covid_level_of_wellbeing'].replace('not provided', np.nan)

md['antibiotic_history'] = md['antibiotic_history'].replace('not provided', np.nan)

md = md.dropna(subset = ['host_body_mass_index', 'antibiotic_history', 'covid_level_of_wellbeing'])

In [78]:
print(f"Total Samples: {len(md)}")
print(md['thdmi_cohort'].value_counts())

Total Samples: 1218
Mexico    497
US        412
UK        309
Name: thdmi_cohort, dtype: int64


In [79]:
import qiime2 as q2 

In [80]:
ft_q2 = q2.Artifact.import_data('FeatureTable[Frequency]', ft)

In [81]:
ft_q2.save('zebra_feature-table.qza')

'zebra_feature-table.qza'

In [49]:
!qiime greengenes2 filter-features \
    --i-feature-table /projects/thdmi/5country/pangenome_filtered/data/zebra_feature-table.qza \
    --i-reference /home/mcdonadt/greengenes2/release/2022.10/2022.10.taxonomy.asv.nwk.qza \
    --o-filtered-feature-table /projects/thdmi/5country/pangenome_filtered/data/gg2_zebra_feature-table.qza

In [82]:
ft_q2 = q2.Artifact.load('/projects/thdmi/5country/pangenome_filtered/data/gg2_zebra_feature-table.qza')
ft = ft_q2.view(pd.DataFrame)

In [83]:
#Calculate total abundance for each sample
ft_ra = ft.copy()
ft_ra['total_abundance'] = ft.sum(axis=1)

In [84]:
# Define the threshold for relative abundance
relative_abundance_threshold = 0.000001  # 0.0001%

In [85]:
ft_ra['total_abundance'] = ft_ra['total_abundance'] * relative_abundance_threshold

In [86]:
# Apply the threshold to each feature to check if it meets the relative abundance threshold
features_above_threshold = ft_ra.iloc[:, :-1].ge(ft_ra['total_abundance'], axis=0)

In [87]:
# Step 1: Calculate the percentage of samples where each feature meets the threshold
feature_prevalence = features_above_threshold.mean(axis=0)

In [88]:
# Step 2: Filter out features with prevalence < 10%
prevalence_threshold = 0.1  # 10%
features_to_keep = feature_prevalence[feature_prevalence >= prevalence_threshold].index

In [89]:
# Filter the original dataframe to keep only these features
filtered_features_df = ft_ra[features_to_keep]

In [90]:
#make sure all samples are above 1000000
filtered_features_df.sum(axis=1).min()

24.0

In [91]:
filtered_features_df = filtered_features_df.loc[filtered_features_df.sum(axis=1) >= 1000000]

In [92]:
md = md.loc[md.index.intersection(filtered_features_df.index)]

In [93]:
print(f"Total Samples: {len(md)}")
print(md['thdmi_cohort'].value_counts())

Total Samples: 1177
Mexico    460
US        410
UK        307
Name: thdmi_cohort, dtype: int64


In [94]:
md = md.rename_axis('#SampleID')

In [95]:
md.to_csv('subsetted_md.tsv', sep = '\t')

In [96]:
filtered_features_df = filtered_features_df.loc[filtered_features_df.index.intersection(md.index)]

In [97]:
filtered_features_df.to_csv('filtered_ft.tsv', sep = '\t')

In [98]:
filtered_features_q2 = q2.Artifact.import_data('FeatureTable[Frequency]', filtered_features_df)

In [99]:
filtered_features_q2.save('filtered_ft.qza')

'filtered_ft.qza'